In [35]:
import os
import yaml
import networkx as nx
from pyvis.network import Network

def load_yaml_files_from_folder(folder_path):
    resources = {
        'deployments': [],
        'services': [],
        'configmaps': []
    }

    # Get all YAML files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.yaml') or file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                documents = yaml.safe_load_all(file)
                for doc in documents:
                    if doc.get('kind') == 'Deployment':
                        resources['deployments'].append(doc)
                    elif doc.get('kind') == 'Service':
                        resources['services'].append(doc)
                    elif doc.get('kind') == 'ConfigMap':
                        resources['configmaps'].append(doc)
    
    return resources

def generate_dependencies(resources):
    dependencies = []

    # Generate dependencies based on Deployment -> Service relationships
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        labels = deployment['spec']['template']['metadata']['labels']
        print(f"Checking Deployment: {deployment_name}")

        # Check if any Service matches the label of the Deployment
        for service in resources['services']:
            service_name = service['metadata']['name']
            if labels.items() <= service['spec'].get('selector', {}).items():
                print(f"  Found Dependency: {deployment_name} -> {service_name}")
                dependencies.append((deployment_name, service_name))

    # Generate dependencies based on ConfigMap usage in Deployment
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        configmap_names = []

        print(f"Checking ConfigMaps for Deployment: {deployment_name}")
        # Check if any ConfigMap is referenced in the Deployment
        for container in deployment['spec']['template']['spec']['containers']:
            if 'envFrom' in container:
                for env_from in container['envFrom']:
                    if 'configMapRef' in env_from:
                        configmap_names.append(env_from['configMapRef']['name'])

        # Create dependencies from Deployment to ConfigMap
        for configmap in resources['configmaps']:
            configmap_name = configmap['metadata']['name']
            if configmap_name in configmap_names:
                print(f"  Found Dependency: {deployment_name} -> {configmap_name}")
                dependencies.append((deployment_name, configmap_name))

    return dependencies

def plot_dependencies_with_pyvis(dependencies, resources):
    # Create a directed graph using networkx
    G = nx.DiGraph()

    # Add nodes for all resources (even if they don't have dependencies)
    for dep in dependencies:
        G.add_edge(dep[0], dep[1])

    # Add all resources as nodes, even if they don't have dependencies
    for deployment in resources['deployments']:
        G.add_node(deployment['metadata']['name'], type='Deployment')
    for service in resources['services']:
        G.add_node(service['metadata']['name'], type='Service')
    for configmap in resources['configmaps']:
        G.add_node(configmap['metadata']['name'], type='ConfigMap')

    # Create a Pyvis network from the NetworkX graph
    net = Network(notebook=True)

    # Load the NetworkX graph into Pyvis
    net.from_nx(G)

    # Show the interactive graph in a browser
    net.show("kubernetes_dependencies_graph.html")

# Example usage:
folder_path = './yaml_files2'  # Path to the folder containing YAML files
resources = load_yaml_files_from_folder(folder_path)
dependencies = generate_dependencies(resources)
plot_dependencies_with_pyvis(dependencies, resources)


Checking Deployment: web-app
  Found Dependency: web-app -> service-loadbalancer
  Found Dependency: web-app -> service-web-app
Checking Deployment: nginx
  Found Dependency: nginx -> service-nginx
Checking Deployment: middleware
  Found Dependency: middleware -> service-middleware
Checking Deployment: db-service
  Found Dependency: db-service -> service-db-service
Checking ConfigMaps for Deployment: web-app
Checking ConfigMaps for Deployment: nginx
Checking ConfigMaps for Deployment: middleware
Checking ConfigMaps for Deployment: db-service
kubernetes_dependencies_graph.html


In [37]:
import os
import yaml
import networkx as nx
from pyvis.network import Network

# Function to load YAML files from a folder
def load_yaml_files_from_folder(folder_path):
    resources = {
        'deployments': [],
        'services': [],
        'configmaps': []
    }

    # Get all YAML files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.yaml') or file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                documents = yaml.safe_load_all(file)
                for doc in documents:
                    if doc.get('kind') == 'Deployment':
                        resources['deployments'].append(doc)
                    elif doc.get('kind') == 'Service':
                        resources['services'].append(doc)
                    elif doc.get('kind') == 'ConfigMap':
                        resources['configmaps'].append(doc)
    
    return resources

# Function to generate dependencies
def generate_dependencies(resources):
    dependencies = []

    # Generate dependencies based on Deployment -> Service relationships
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        labels = deployment['spec']['template']['metadata']['labels']

        # Check if any Service matches the label of the Deployment
        for service in resources['services']:
            service_name = service['metadata']['name']
            if labels.items() <= service['spec'].get('selector', {}).items():
                dependencies.append((deployment_name, service_name))

    # Generate dependencies based on ConfigMap usage in Deployment
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        configmap_names = []

        # Check if any ConfigMap is referenced in the Deployment
        for container in deployment['spec']['template']['spec']['containers']:
            if 'envFrom' in container:
                for env_from in container['envFrom']:
                    if 'configMapRef' in env_from:
                        configmap_names.append(env_from['configMapRef']['name'])

        # Create dependencies from Deployment to ConfigMap
        for configmap in resources['configmaps']:
            configmap_name = configmap['metadata']['name']
            if configmap_name in configmap_names:
                dependencies.append((deployment_name, configmap_name))

    return dependencies

# Function to plot dependencies with Pyvis
def plot_dependencies_with_pyvis(dependencies, resources):
    # Create a directed graph using networkx
    G = nx.DiGraph()

    # Add nodes for all resources (even if they don't have dependencies)
    for dep in dependencies:
        G.add_edge(dep[0], dep[1])

    # Create the Pyvis network object
    net = Network(notebook=True)

    # Add nodes with additional information (IP, Port, Labels, etc.)
    for service in resources['services']:
        service_name = service['metadata']['name']
        ip = service.get('status', {}).get('loadBalancer', {}).get('ingress', [{}])[0].get('ip', 'N/A')
        ports = ", ".join([str(port['port']) for port in service.get('spec', {}).get('ports', [])])
        selector = ", ".join([f"{key}={value}" for key, value in service['spec'].get('selector', {}).items()])
        
        # Add the service node with additional info
        net.add_node(service_name, title=f"IP: {ip}\nPorts: {ports}\nSelector: {selector}", label=service_name)
        
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        replicas = deployment['spec'].get('replicas', 'N/A')
        containers = ", ".join([container['name'] for container in deployment['spec']['template']['spec']['containers']])

        # Add the deployment node with additional info
        net.add_node(deployment_name, title=f"Replicas: {replicas}\nContainers: {containers}", label=deployment_name)

    for configmap in resources['configmaps']:
        configmap_name = configmap['metadata']['name']
        
        # Add the ConfigMap node with additional info
        net.add_node(configmap_name, title=f"ConfigMap Name: {configmap_name}", label=configmap_name)

    # Add edges (dependencies) to the graph
    for dep in dependencies:
        net.add_edge(dep[0], dep[1])

    # Show the interactive graph in a browser
    net.show("kubernetes_dependencies_graph_with_details.html")

# Example usage:
folder_path = './yaml_files2'  # Path to the folder containing YAML files
resources = load_yaml_files_from_folder(folder_path)
dependencies = generate_dependencies(resources)
plot_dependencies_with_pyvis(dependencies, resources)


kubernetes_dependencies_graph_with_details.html


In [39]:
import os
import yaml
import networkx as nx
from pyvis.network import Network

# Function to load YAML files from a folder
def load_yaml_files_from_folder(folder_path):
    resources = {
        'deployments': [],
        'services': [],
        'configmaps': []
    }

    # Get all YAML files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.yaml') or file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                documents = yaml.safe_load_all(file)
                for doc in documents:
                    if doc.get('kind') == 'Deployment':
                        resources['deployments'].append(doc)
                    elif doc.get('kind') == 'Service':
                        resources['services'].append(doc)
                    elif doc.get('kind') == 'ConfigMap':
                        resources['configmaps'].append(doc)
    
    return resources

# Function to generate dependencies
def generate_dependencies(resources):
    dependencies = []

    # Generate dependencies based on Deployment -> Service relationships
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        labels = deployment['spec']['template']['metadata']['labels']

        # Check if any Service matches the label of the Deployment
        for service in resources['services']:
            service_name = service['metadata']['name']
            if labels.items() <= service['spec'].get('selector', {}).items():
                dependencies.append((deployment_name, service_name))

    # Generate dependencies based on ConfigMap usage in Deployment
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        configmap_names = []

        # Check if any ConfigMap is referenced in the Deployment
        for container in deployment['spec']['template']['spec']['containers']:
            if 'envFrom' in container:
                for env_from in container['envFrom']:
                    if 'configMapRef' in env_from:
                        configmap_names.append(env_from['configMapRef']['name'])

        # Create dependencies from Deployment to ConfigMap
        for configmap in resources['configmaps']:
            configmap_name = configmap['metadata']['name']
            if configmap_name in configmap_names:
                dependencies.append((deployment_name, configmap_name))

    return dependencies

# Function to plot dependencies with Pyvis
def plot_dependencies_with_pyvis(dependencies, resources):
    # Create a directed graph using networkx
    G = nx.DiGraph()

    # Add nodes for all resources (even if they don't have dependencies)
    for dep in dependencies:
        G.add_edge(dep[0], dep[1])

    # Create the Pyvis network object
    net = Network(notebook=True)

    # Add nodes with additional information (IP, Port, Labels, etc.)
    for service in resources['services']:
        service_name = service['metadata']['name']
        ip = service.get('status', {}).get('loadBalancer', {}).get('ingress', [{}])[0].get('ip', 'N/A')
        
        # Construct dynamic ports information
        ports_info = []
        for port in service.get('spec', {}).get('ports', []):
            port_details = ", ".join([f"{key}: {port[key]}" for key in port.keys()])
            ports_info.append(port_details)
        
        ports = "\n".join(ports_info) if ports_info else 'N/A'
        
        selector = ", ".join([f"{key}={value}" for key, value in service['spec'].get('selector', {}).items()])
        
        # Add the service node with dynamic port info
        net.add_node(service_name, title=f"IP: {ip}\nPorts: {ports}\nSelector: {selector}", label=service_name)
        
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        replicas = deployment['spec'].get('replicas', 'N/A')
        containers = ", ".join([container['name'] for container in deployment['spec']['template']['spec']['containers']])

        # Add the deployment node with additional info
        net.add_node(deployment_name, title=f"Replicas: {replicas}\nContainers: {containers}", label=deployment_name)

    for configmap in resources['configmaps']:
        configmap_name = configmap['metadata']['name']
        
        # Add the ConfigMap node with additional info
        net.add_node(configmap_name, title=f"ConfigMap Name: {configmap_name}", label=configmap_name)

    # Add edges (dependencies) to the graph
    for dep in dependencies:
        net.add_edge(dep[0], dep[1])

    # Show the interactive graph in a browser
    net.show("kubernetes_dependencies_graph_with_dynamic_details.html")

# Example usage:
folder_path = './yaml_files2'  # Path to the folder containing YAML files
resources = load_yaml_files_from_folder(folder_path)
dependencies = generate_dependencies(resources)
plot_dependencies_with_pyvis(dependencies, resources)


kubernetes_dependencies_graph_with_dynamic_details.html


In [41]:
import os
import yaml
import networkx as nx
from pyvis.network import Network

# Function to load YAML files from a folder
def load_yaml_files_from_folder(folder_path):
    resources = {
        'deployments': [],
        'services': [],
        'configmaps': []
    }

    # Get all YAML files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.yaml') or file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                documents = yaml.safe_load_all(file)
                for doc in documents:
                    if doc.get('kind') == 'Deployment':
                        resources['deployments'].append(doc)
                    elif doc.get('kind') == 'Service':
                        resources['services'].append(doc)
                    elif doc.get('kind') == 'ConfigMap':
                        resources['configmaps'].append(doc)
    
    return resources

# Function to generate dependencies
def generate_dependencies(resources):
    dependencies = []

    # Generate dependencies based on Deployment -> Service relationships
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        labels = deployment['spec']['template']['metadata']['labels']

        # Check if any Service matches the label of the Deployment
        for service in resources['services']:
            service_name = service['metadata']['name']
            if labels.items() <= service['spec'].get('selector', {}).items():
                dependencies.append((deployment_name, service_name))

    # Generate dependencies based on ConfigMap usage in Deployment
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        configmap_names = []

        # Check if any ConfigMap is referenced in the Deployment
        for container in deployment['spec']['template']['spec']['containers']:
            if 'envFrom' in container:
                for env_from in container['envFrom']:
                    if 'configMapRef' in env_from:
                        configmap_names.append(env_from['configMapRef']['name'])

        # Create dependencies from Deployment to ConfigMap
        for configmap in resources['configmaps']:
            configmap_name = configmap['metadata']['name']
            if configmap_name in configmap_names:
                dependencies.append((deployment_name, configmap_name))

    return dependencies

# Function to plot dependencies with Pyvis
def plot_dependencies_with_pyvis(dependencies, resources):
    # Create a directed graph using networkx
    G = nx.DiGraph()

    # Add nodes for all resources (even if they don't have dependencies)
    for dep in dependencies:
        G.add_edge(dep[0], dep[1])

    # Create the Pyvis network object
    net = Network(notebook=True)

    # Function to recursively extract details from YAML (handle nested structures)
    def extract_details(data):
        if isinstance(data, dict):
            return "\n".join([f"{key}: {extract_details(value)}" for key, value in data.items()])
        elif isinstance(data, list):
            return ", ".join([extract_details(item) for item in data])
        else:
            return str(data)

    # Add service nodes with dynamically extracted fields
    for service in resources['services']:
        service_name = service['metadata']['name']
        details = extract_details(service)
        net.add_node(service_name, title=details, label=service_name)
        
    # Add deployment nodes with dynamically extracted fields
    for deployment in resources['deployments']:
        deployment_name = deployment['metadata']['name']
        details = extract_details(deployment)
        net.add_node(deployment_name, title=details, label=deployment_name)

    # Add ConfigMap nodes with dynamically extracted fields
    for configmap in resources['configmaps']:
        configmap_name = configmap['metadata']['name']
        details = extract_details(configmap)
        net.add_node(configmap_name, title=details, label=configmap_name)

    # Add edges (dependencies) to the graph
    for dep in dependencies:
        net.add_edge(dep[0], dep[1])

    # Show the interactive graph in a browser
    net.show("kubernetes_dependencies_graph_with_dynamic_details.html")

# Example usage:
folder_path = './yaml_files2'  # Path to the folder containing YAML files
resources = load_yaml_files_from_folder(folder_path)
dependencies = generate_dependencies(resources)
plot_dependencies_with_pyvis(dependencies, resources)


kubernetes_dependencies_graph_with_dynamic_details.html
